# Springboard Capstone Project #2: 
## Vacation Rental Price Recommender

- **Data source:** A sample of vacation rental (VR) listings from AirDNA
- **Rental market being analyzed:** Cape Coral, FL

## Package & Data Source Imports

In [1]:
import pandas as pd
import json

In [2]:
# read in VR data json file into a DataFrame

# didn't work: vr_data = pd.read_json('../vr_datasets/output.json')

data = json.load(open('../vr_datasets/output.json'))
df = pd.json_normalize(data)

df

,listings_limited,properties,listings_returned,request_info.currency,area_info.geom.code.country,area_info.geom.code.state,area_info.geom.code.region,area_info.geom.code.city,area_info.geom.name.country,area_info.geom.name.state,area_info.geom.name.region,area_info.geom.name.city,area_info.geom.id.country,area_info.geom.id.state,area_info.geom.id.region,area_info.geom.id.city
0,False,"[{'airbnb_property_id': 11046205, 'rating': 0....",1275,native,us,florida,33914,cape-coral,United States,Florida,33914,Cape Coral,1,11,2833,60018


### Now that we've got the data loaded in as a DF and flattened most of the initial nested dictionary structure, let's subset the Properties data into it's own DF. 

In [3]:
properties = df['properties'].to_frame()
print(type(properties))
properties

<class 'pandas.core.frame.DataFrame'>


,properties
0,"[{'airbnb_property_id': 11046205, 'rating': 0...."


### I'm not sure the best way to proceed from how I've parsed the original data file thus far. Going to try another approach for extracting the data...

In [4]:
with open('../vr_datasets/output.json','r') as f:
    data = json.loads(f.read())
    
properties_df = pd.json_normalize(data, record_path=['properties'])

properties_df

,airbnb_property_id,rating,property_type,bathrooms,title,occ,bedrooms,accommodates,adr,img_cover,...,homeaway_property_id,longitude,latitude,m_homeaway_property_id,days_available,id,room_type,regions.zipcode_ids,platforms.airbnb_property_id,platforms.homeaway_property_id
0,11046205,0.0,Villa,2.0,Luxurious Pool Villa with private boat slip,0.404494,3,2,208.3330,https://a0.muscache.com/im/pictures/375a2880-0...,...,None,-82.033230,26.558650,21941ha,89,1753173,Entire home/apt,[2833],11046205.0,21941ha
1,9786136,9.8,House,1.0,Private Friendly Oasis,0.345833,2,4,80.7309,https://a0.muscache.com/im/pictures/a141200a-2...,...,None,-81.995120,26.586620,None,240,1556362,Private room,[2833],9786136.0,None
2,1003435283,10.0,house,2.0,Paradise - New Pool - Canal and Sunshine,0.824742,3,6,136.7500,https://media.vrbo.com/lodging/35000000/340800...,...,1494515,-81.976502,26.544985,None,97,1003435283,Entire home/apt,[2833],NaN,1494515
3,7836717,9.4,House,3.0,7th Heaven - Unique Home Perfect for 2 Families,0.605634,4,8,233.5120,https://a0.muscache.com/im/pictures/03d129d9-8...,...,None,-82.019160,26.571760,9247875ha,71,501312,Entire home/apt,[2833],7836717.0,9247875ha
4,9893241,10.0,Guest suite,1.0,Cape Paradise - Poolside Cabana,0.152381,1,2,189.6940,https://a0.muscache.com/im/pictures/3f0c011f-8...,...,None,-82.004250,26.564580,None,105,1556359,Private room,[2833],9893241.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,1000742730,10.0,house,2.0,Beautiful Gulf Access Vacation Home - Pool - H...,0.625000,3,6,238.0000,https://media.vrbo.com/lodging/40000000/399500...,...,872964,-81.977301,26.566971,None,40,1000742730,Entire home/apt,[2833],NaN,872964
1271,1000375573,10.0,House,2.0,"Waterfront Home with heated Pool and Spa, Gulf...",0.495868,3,6,184.4800,https://media.vrbo.com/lodging/35000000/340200...,...,4312377ha,-81.977324,26.582227,None,242,1000375573,Entire home/apt,[2833],NaN,4312377ha
1272,1001904447,10.0,villa,3.5,"Luxury Surfside Home With Waterview, Heated Po...",0.265625,3,6,366.9620,https://media.vrbo.com/lodging/34000000/335700...,...,4760935ha,-82.035941,26.589387,None,128,1001904447,Entire home/apt,[2833],NaN,4760935ha
1273,29833242,8.7,House,3.0,Roelens Vacations - Villa Hideaway - Cape Coral,0.314024,3,6,264.1110,https://a0.muscache.com/im/pictures/a6974f6a-1...,...,None,-82.014170,26.595750,7433783ha,328,11541379,Entire home/apt,[2833],29833242.0,7433783ha


### This is definitely much closer to what I am wanting to work with. 

After two different approaches to loading in the data, my latter approach seems to have been the better route. Based on my research, I can still add other metadata from the original JSON file that I want to include, by passing in a list of keys to the 'meta' parameter of 'json_normalize'. 

In [5]:
properties_df = pd.json_normalize(data, record_path=['properties'], 
                                  meta=['area_info.geom.name.country', 'area_info.geom.name.state', 
                                        'area_info.geom.name.region', 'area_info.geom.name.city'])

properties_df

KeyError: "Try running with errors='ignore' as key 'area_info.geom.name.country' is not always present"

### Based on the error above, going to try same code again but this time will add in errors='ignore'

In [6]:
properties_df = pd.json_normalize(data, record_path=['properties'], 
                                  meta=['area_info.geom.name.country', 'area_info.geom.name.state', 
                                        'area_info.geom.name.region', 'area_info.geom.name.city'],
                                 errors='ignore')

properties_df

,airbnb_property_id,rating,property_type,bathrooms,title,occ,bedrooms,accommodates,adr,img_cover,...,days_available,id,room_type,regions.zipcode_ids,platforms.airbnb_property_id,platforms.homeaway_property_id,area_info.geom.name.country,area_info.geom.name.state,area_info.geom.name.region,area_info.geom.name.city
0,11046205,0.0,Villa,2.0,Luxurious Pool Villa with private boat slip,0.404494,3,2,208.3330,https://a0.muscache.com/im/pictures/375a2880-0...,...,89,1753173,Entire home/apt,[2833],11046205.0,21941ha,NaN,NaN,NaN,NaN
1,9786136,9.8,House,1.0,Private Friendly Oasis,0.345833,2,4,80.7309,https://a0.muscache.com/im/pictures/a141200a-2...,...,240,1556362,Private room,[2833],9786136.0,None,NaN,NaN,NaN,NaN
2,1003435283,10.0,house,2.0,Paradise - New Pool - Canal and Sunshine,0.824742,3,6,136.7500,https://media.vrbo.com/lodging/35000000/340800...,...,97,1003435283,Entire home/apt,[2833],NaN,1494515,NaN,NaN,NaN,NaN
3,7836717,9.4,House,3.0,7th Heaven - Unique Home Perfect for 2 Families,0.605634,4,8,233.5120,https://a0.muscache.com/im/pictures/03d129d9-8...,...,71,501312,Entire home/apt,[2833],7836717.0,9247875ha,NaN,NaN,NaN,NaN
4,9893241,10.0,Guest suite,1.0,Cape Paradise - Poolside Cabana,0.152381,1,2,189.6940,https://a0.muscache.com/im/pictures/3f0c011f-8...,...,105,1556359,Private room,[2833],9893241.0,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,1000742730,10.0,house,2.0,Beautiful Gulf Access Vacation Home - Pool - H...,0.625000,3,6,238.0000,https://media.vrbo.com/lodging/40000000/399500...,...,40,1000742730,Entire home/apt,[2833],NaN,872964,NaN,NaN,NaN,NaN
1271,1000375573,10.0,House,2.0,"Waterfront Home with heated Pool and Spa, Gulf...",0.495868,3,6,184.4800,https://media.vrbo.com/lodging/35000000/340200...,...,242,1000375573,Entire home/apt,[2833],NaN,4312377ha,NaN,NaN,NaN,NaN
1272,1001904447,10.0,villa,3.5,"Luxury Surfside Home With Waterview, Heated Po...",0.265625,3,6,366.9620,https://media.vrbo.com/lodging/34000000/335700...,...,128,1001904447,Entire home/apt,[2833],NaN,4760935ha,NaN,NaN,NaN,NaN
1273,29833242,8.7,House,3.0,Roelens Vacations - Villa Hideaway - Cape Coral,0.314024,3,6,264.1110,https://a0.muscache.com/im/pictures/a6974f6a-1...,...,328,11541379,Entire home/apt,[2833],29833242.0,7433783ha,NaN,NaN,NaN,NaN


### Hmm... I didn't expect NaNs for each of the 'meta' values. After further researching, it looks like I may have had the formatting wrong for accessing nested keys in the 'meta' parameter..

In [7]:
properties_df = pd.json_normalize(data, record_path=['properties'], 
                                  meta=[['area_info', 'geom', 'name', 'country'],
                                       ['area_info', 'geom', 'name', 'state'],
                                       ['area_info', 'geom', 'name', 'region'],
                                       ['area_info', 'geom', 'name', 'city']],
                                 errors='ignore')

properties_df

,airbnb_property_id,rating,property_type,bathrooms,title,occ,bedrooms,accommodates,adr,img_cover,...,days_available,id,room_type,regions.zipcode_ids,platforms.airbnb_property_id,platforms.homeaway_property_id,area_info.geom.name.country,area_info.geom.name.state,area_info.geom.name.region,area_info.geom.name.city
0,11046205,0.0,Villa,2.0,Luxurious Pool Villa with private boat slip,0.404494,3,2,208.3330,https://a0.muscache.com/im/pictures/375a2880-0...,...,89,1753173,Entire home/apt,[2833],11046205.0,21941ha,United States,Florida,33914,Cape Coral
1,9786136,9.8,House,1.0,Private Friendly Oasis,0.345833,2,4,80.7309,https://a0.muscache.com/im/pictures/a141200a-2...,...,240,1556362,Private room,[2833],9786136.0,None,United States,Florida,33914,Cape Coral
2,1003435283,10.0,house,2.0,Paradise - New Pool - Canal and Sunshine,0.824742,3,6,136.7500,https://media.vrbo.com/lodging/35000000/340800...,...,97,1003435283,Entire home/apt,[2833],NaN,1494515,United States,Florida,33914,Cape Coral
3,7836717,9.4,House,3.0,7th Heaven - Unique Home Perfect for 2 Families,0.605634,4,8,233.5120,https://a0.muscache.com/im/pictures/03d129d9-8...,...,71,501312,Entire home/apt,[2833],7836717.0,9247875ha,United States,Florida,33914,Cape Coral
4,9893241,10.0,Guest suite,1.0,Cape Paradise - Poolside Cabana,0.152381,1,2,189.6940,https://a0.muscache.com/im/pictures/3f0c011f-8...,...,105,1556359,Private room,[2833],9893241.0,None,United States,Florida,33914,Cape Coral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,1000742730,10.0,house,2.0,Beautiful Gulf Access Vacation Home - Pool - H...,0.625000,3,6,238.0000,https://media.vrbo.com/lodging/40000000/399500...,...,40,1000742730,Entire home/apt,[2833],NaN,872964,United States,Florida,33914,Cape Coral
1271,1000375573,10.0,House,2.0,"Waterfront Home with heated Pool and Spa, Gulf...",0.495868,3,6,184.4800,https://media.vrbo.com/lodging/35000000/340200...,...,242,1000375573,Entire home/apt,[2833],NaN,4312377ha,United States,Florida,33914,Cape Coral
1272,1001904447,10.0,villa,3.5,"Luxury Surfside Home With Waterview, Heated Po...",0.265625,3,6,366.9620,https://media.vrbo.com/lodging/34000000/335700...,...,128,1001904447,Entire home/apt,[2833],NaN,4760935ha,United States,Florida,33914,Cape Coral
1273,29833242,8.7,House,3.0,Roelens Vacations - Villa Hideaway - Cape Coral,0.314024,3,6,264.1110,https://a0.muscache.com/im/pictures/a6974f6a-1...,...,328,11541379,Entire home/apt,[2833],29833242.0,7433783ha,United States,Florida,33914,Cape Coral


### Much better! There is still some cleaning work to be done, but I think there are some interesting features here to work with for analysis and modeling. 